In [1]:
# set up path to facenet_pytorch_c
import sys
sys.path.insert(1, '/home/ubuntu/mtcnn')

In [2]:
# facenet_pytorch_c: avoid confusion with system default facenet_pytorch
from facenet_pytorch_c import MTCNN

from tqdm import tqdm
import numpy as np
import os

# pytorch
import torch
import torch.optim as optim
from torch import nn

# data handling
from torch.utils.data import DataLoader

# torchvision libs
from torchvision import datasets
from torchvision import transforms

# other custom scripts
import utils

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Available device: " + str(device))

# training hyperparameters
learning_rate = 1e-3
epochs = 30
decay_step = [15]
decay_rate = 0.1
opt = 'Adam'    # either Adam or SGD
batch_size = 64


Available device: cuda:0


In [4]:
# data loading parameters
workers = 4
resize_shape = (224, 224)

In [5]:
# get data
x_train, age_train, fn_train, bbox_train, prob_train, land_train, x_valid, age_valid, fn_valid, bbox_valid, prob_valid, land_valid = utils.get_images(
    r'/home/ubuntu/UTKFace', resize_shape=resize_shape
    )

100%|██████████| 23708/23708 [04:37<00:00, 85.39it/s]

Ignored: 
18_0_1_20170113175821404.jpg.chip.jpg
18_1_0_20170109213103958.jpg.chip.jpg
1_0_0_20170109193052283.jpg.chip.jpg
1_0_0_20170109194120301.jpg.chip.jpg
1_1_0_20170109190852630.jpg.chip.jpg
1_1_0_20170109194452834.jpg.chip.jpg
1_1_4_20161221202159241.jpg.chip.jpg
1_1_4_20170109194502921.jpg.chip.jpg
20_1_0_20170109214125992.jpg.chip.jpg
20_1_3_20170119180826861.jpg.chip.jpg
25_0_1_20170116001410569.jpg.chip.jpg
25_0_4_20170117155154894.jpg.chip.jpg
25_1_3_20170119171206287.jpg.chip.jpg
26_0_0_20170113210126964.jpg.chip.jpg
26_0_0_20170113210319560.jpg.chip.jpg
26_0_0_20170113210605342.jpg.chip.jpg
26_0_2_20170119180202654.jpg.chip.jpg
26_1_1_20170116153459393.jpg.chip.jpg
26_1_1_20170117201147190.jpg.chip.jpg
26_1_2_20170116180208093.jpg.chip.jpg
28_0_1_20170117020012900.jpg.chip.jpg
28_1_2_20170116164703724.jpg.chip.jpg
2_1_2_20161219142349633.jpg.chip.jpg
2_1_2_20161219203118564.jpg.chip.jpg
30_1_2_20170105162719051.jpg.chip.jpg
31_1_0_20170117181946996.jpg.chip.jpg
33_0_0_201

In [6]:
# setup mtcnn
"""
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, keep_all=True,
    device=device
)
"""
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.5, 0.7, 0.7], factor=0.709, post_process=True, keep_all=True,
    device=device
)

In [ ]:
# define data reader

# no need to convert to PIL, because get_images already does that

transform_train = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])


"""
transform_valid = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.ToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
"""

train_reader = DataLoader(
    utils.UTKDataLoader(x_train, y_train, tsfm=transform_train),
    batch_size=batch_size, num_workers=workers, shuffle=True
    )

"""
valid_reader = DataLoader(
    utils.UTKDataLoader(x_valid, y_valid, tsfm=transform_valid),
    batch_size=batch_size, num_workers=workers, shuffle=False
    )
"""